# Capstone Notebook
#### IBM Data Science Capstone Project on Coursera

## Import/Install Required Libraries

In [1]:
import pandas as pd
pd.set_option('max_rows', 200)
import numpy as np
! pip install geocoder
import geocoder as gc
import requests
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0
import folium
import random
import matplotlib.cm as cm
import matplotlib.colors as colors
print('--Imports Completed--')

--Imports Completed--


## Los Angeles County Zipcodes

#### Retrieve/Format Table of CA Zip Codes

In [24]:
#Download & Format Available California Zipcodes
url = 'https://www.geonames.org/postal-codes/US/CA/california.html'
laData = pd.read_html(url)
laData = laData[2]  #Select Correct Table from url
laData
laData = laData.drop(laData.columns[0],axis=1) 
laData = laData.drop(laData.columns[5:len(laData.columns)],axis=1).reset_index(drop=True) #Drop extraneous columns
#Rename Columns
cols = list(laData.columns)
cols[3] = 'State'
cols[4] = 'County'
laData.columns = cols
laData = laData.drop(len(laData.index)-1).reset_index(drop=True) #Drop extraneous row (last row)

In [25]:
#extract coordinates fromm odd rows 
#reinsert as columns 
lat = [] 
long = [] 
for count, n in enumerate(laData['Place']):
    if (count % 2 != 0):
        temp = n.split('/')
        lat.append(temp[0])
        long.append(temp[1])
        laData = laData.drop(count)
laData = laData.reset_index(drop=True)

#add Latitude and Logitude to LA Dataframe
data = {'Latitude':lat,'Longitude':long}
temp = pd.DataFrame(data,dtype='float')
laData = laData.join(temp)
laData = laData[laData['County'].str.contains('Los Angeles')] #Select only zip codes in Los Angeles

#### Remove Zipcodes with Duplicate Coordinates

In [28]:
laData = laData.sort_values(by = ['Latitude','Longitude']).reset_index(drop=True)

for (row) in range(len(laData.index)-1):
    if laData.loc[row,'Latitude'] == laData.loc[row+1,'Latitude']:
        if laData.loc[row,'Longitude'] == laData.loc[row+1,'Longitude']:
            laData = laData.drop(row)

laData = laData.sort_values('Code').reset_index(drop=True) #resort by zipcode

#### Complete LA County Zip Codes

In [29]:
#Display Corrected Dataframe Shape & Head
print('Rows:',laData.shape[0],'\nColumns:',laData.shape[1])
laData

Rows: 123 
Columns: 7


,Place,Code,Country,State,County,Latitude,Longitude
0,Los Angeles,90001,United States,California,Los Angeles,33.973,-118.248
1,Los Angeles,90002,United States,California,Los Angeles,33.950,-118.246
2,Los Angeles,90003,United States,California,Los Angeles,33.965,-118.273
3,Los Angeles,90004,United States,California,Los Angeles,34.076,-118.303
4,Los Angeles,90005,United States,California,Los Angeles,34.059,-118.301
5,Los Angeles,90006,United States,California,Los Angeles,34.049,-118.292
6,Los Angeles,90007,United States,California,Los Angeles,34.029,-118.287
7,Los Angeles,90008,United States,California,Los Angeles,34.012,-118.341
8,Los Angeles,90010,United States,California,Los Angeles,34.061,-118.303
9,Los Angeles,90011,United States,California,Los Angeles,34.008,-118.258


## Map of Zip Code Locations

In [6]:
#Get Coordinates for Map
address = 'Los Angeles, CA'
geolocator = Nominatim(user_agent="LA_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(address,'\nLatitude:',latitude,'\nLongitude:',longitude)


Los Angeles, CA 
Latitude: 34.0536909 
Longitude: -118.2427666


In [32]:
#Create and Display Map
laMap = folium.Map(location=[latitude, longitude],tiles = 'Stamen Toner', zoom_start=10)

#add Zipcode labels to map
for lat, lng, code, place in zip(laData['Latitude'], laData['Longitude'], laData['Code'], laData['Place']):
    label = '{}: {}'.format(place, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.5,
        parse_html=False).add_to(laMap)

#display map
laMap

## Retrieve Venues via Foursquare API

In [37]:
# @hidden_cell
#Define Foursquare Credentials
CLIENT_ID = 'V5GTPEARUI4LNNVRL2Z2CBTSKQX32TZB14ZUP4KKDB0DS01L' #Foursquare ID
CLIENT_SECRET = 'FNNRQJQGSE23SQVTQ1MX0DKED01IW5B5O50H0R5K3CKFEBWL' #Foursquare Secret
VERSION = '20180605'

In [9]:
#Function to Create Dataframe of Venues from Foursquare API
def getVenues(code, latitudes, longitudes, radius=500):
    
    venues_list=[]
    dropList = []  
    for code, lat, lng in zip(code, latitudes, longitudes):
        print(code)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        if results == []:
            print('Warning no venues returned for: ',code)
            dropList.append(code)
        # return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']  
    return(nearby_venues, dropList)

In [10]:
laVenues, laDrop = getVenues(laData['Code'],laData['Latitude'],laData['Longitude'])

90001
90002
90003
90004
90005
90006
90007
90008
90010
90011
90012
90013
90014
90015
90016
90017
90018
90019
90020
90021
90022
90023
90024
90025
90026
90027
90028
90029
90031
90032
90033
90034
90035
90036
90037
90038
90039
90040
90041
90042
90043
90044
90045
90046
90047
90048
90049
90056
90057
90058
90059
90061
Warning no venues returned for:  90061
90062
90063
90064
90065
90066
90067
90068
90069
90071
90077
90096
90210
90211
90212
90213
90220
90221
90223
90230
90232
90233
90239
90240
90241
90242
90245
90247
90248
90249
90250
90251
90254
90255
90260
90261
90262
90264
90265
Warning no venues returned for:  90265
90266
90270
90272
90274
90275
90277
90278
90280
90290
90291
90292
90293
90294
90301
90302
90303
90304
90305
90310
90312
90401
90402
90403
90404
90405
90407
90503
90802
90804
91006
91105
91107
91505


In [11]:
#Display Venues
laVenues.head()

,Zipcode,Zipcode Latitude,Zipcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,90001,33.973,-118.248,Superior Grocers,33.973280,-118.247079,Grocery Store
1,90001,33.973,-118.248,Rite Aid,33.974383,-118.246351,Pharmacy
2,90001,33.973,-118.248,Bill's Drive In,33.974500,-118.244225,Burger Joint
3,90001,33.973,-118.248,SUBWAY,33.975311,-118.248038,Sandwich Place
4,90001,33.973,-118.248,Jack in the Box,33.975167,-118.250313,Fast Food Restaurant


In [12]:
#Create Dataframe for kClusters Analysis
laEncode = pd.get_dummies(laVenues[['Venue Category']], prefix="", prefix_sep="")
laEncode['Zipcode'] = laVenues['Zipcode']
fixed_columns = [laEncode.columns[-1]] + list(laEncode.columns[:-1])
laEncode = laEncode[fixed_columns]
laSorted = laEncode.groupby(by='Zipcode').mean().reset_index()
laCluster = laSorted.drop('Zipcode', axis=1)

In [13]:
#Show mean values of category types sorted by zipcode
laSorted.head(10)

,Zipcode,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Watch Shop,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Yoshoku Restaurant
0,90001,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,90002,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,90003,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
3,90004,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,90005,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.025641,0.0,0.000000,0.0,0.025641,0.0
5,90006,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
6,90007,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.040000,0.0
7,90008,0.0,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.032258,0.0,0.000000,0.0
8,90010,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.019231,0.0,0.000000,0.0,0.019231,0.0
9,90011,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [14]:
#Create DataFrame of Top 5 Venue Categories for each Zipcode
topVen = pd.DataFrame()
#Get top 5 venues for each Zipcode
for count, z in enumerate(laSorted['Zipcode']):
    temp = laSorted[laSorted['Zipcode'] == z]  #Select all of a single zipcode
    temp = temp.drop('Zipcode',axis=1)
    temp.reset_index()
    temp = temp.sort_values(by=count,axis=1,ascending=False)
    topVen = topVen.append(pd.Series(temp.iloc[:,0:5].columns),ignore_index = True)
    
#Rename Columns & Display first 5 rows
topVen.columns = ['#1 Venue','#2 Venue','#3 Venue','#4 Venue','#5 Venue'] 
print(topVen.shape)
topVen.head()

(121, 5)


,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,Fruit & Vegetable Store,Burger Joint,Shoe Store,Grocery Store,Donut Shop
1,Park,ATM,Optical Shop,North Indian Restaurant,Noodle House
2,Taco Place,ATM,Nail Salon,North Indian Restaurant,Noodle House
3,Mexican Restaurant,Convenience Store,Cocktail Bar,Sushi Restaurant,Pizza Place
4,Korean Restaurant,Karaoke Bar,Coffee Shop,Café,Hotel


In [15]:
#Fit Data
k = 6
kmeans = KMeans(k, random_state=0).fit(laCluster)

In [16]:
#add Cluster Labels to Top Venues Dataframe
topVen.insert(0,'Cluster Labels',kmeans.labels_ + 1)

In [17]:
# Join Zipcodes & Coordinates with Cluster Labels and Top Venues
laFinal = pd.DataFrame()
laFinal = laData.loc[:,['Code','Place','Latitude','Longitude']].sort_values(by='Code').reset_index(drop=True)
for x in laDrop:
    if laFinal['Code'].str.contains(x).any():
        laFinal = laFinal.drop(laFinal.index[laFinal['Code']==x])
laFinal = laFinal.reset_index(drop=True)
laFinal = laFinal.join(topVen)
laFinal = laFinal.sort_values('Cluster Labels').reset_index(drop=True)

In [18]:
##### initialize map
clusterMap = folium.Map(location=[latitude, longitude],tiles='Stamen Toner', zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**4 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(laFinal['Latitude'], laFinal['Longitude'], laFinal['Code'], laFinal['Cluster Labels']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=500,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.75).add_to(clusterMap)
print('K    Count\n-----------')
print(laFinal['Cluster Labels'].value_counts(sort=False))
clusterMap

K    Count
-----------
1     7
2    69
3     2
4     1
5     2
6    40
Name: Cluster Labels, dtype: int64


In [19]:
laFinal

,Code,Place,Latitude,Longitude,Cluster Labels,#1 Venue,#2 Venue,#3 Venue,#4 Venue,#5 Venue
0,90077,Los Angeles,34.111,-118.450,1,Scenic Lookout,Art Gallery,Home Service,ATM,New American Restaurant
1,90264,Malibu,34.005,-118.810,1,Beach,Scenic Lookout,Art Gallery,ATM,New American Restaurant
2,90275,Rancho Palos Verdes,33.752,-118.367,1,Trail,Furniture / Home Store,Sporting Goods Shop,Beach,Scenic Lookout
3,90210,Beverly Hills,34.090,-118.406,1,Park,Art Gallery,Tennis Court,ATM,Neighborhood
4,90040,Los Angeles,33.991,-118.153,1,Coffee Shop,ATM,Neighborhood,Office,North Indian Restaurant
5,90402,Santa Monica,34.035,-118.503,1,Beach,Trail,ATM,Neighborhood,North Indian Restaurant
6,90058,Los Angeles,33.999,-118.213,1,Sandwich Place,Food,ATM,Neighborhood,North Indian Restaurant
7,90241,Downey,33.942,-118.131,2,Mexican Restaurant,Asian Restaurant,Chinese Restaurant,Bar,Burger Joint
8,90240,Downey,33.958,-118.117,2,Gym / Fitness Center,BBQ Joint,ATM,New American Restaurant,Office
9,90239,Downey,33.940,-118.133,2,Mexican Restaurant,Asian Restaurant,Burger Joint,Chinese Restaurant,Bar
